# Spark Job Example
* Requires you to have a spark master running
* Requires you to have aws configured to allow pushing files to S3

In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
import disdat.api as api
from disdat.api import Bundle
import pandas as pd
import pickle
import time
import luigi
from pyspark.sql import SparkSession
from pyspark import SparkConf
import os

%aimport pipelines.spark_tasks

# Make a bundle with s3 paths
NOTE: Requires a remote context to push to 

In [4]:
data_context = 'example-context'
remote_context_url = 's3://disdat-cdo-prd/'  # <------ Replace with the location of your Disdat contexts on S3
api.context(data_context)
api.remote(data_context, data_context, remote_context_url)

with Bundle(data_context, name="s3_files") as b:
    f1 = b.get_file("file_1.txt")
    with open(f1, mode='w') as f:
        f.write("This is our first file!")
    b.add_data(f1)

# Push and remove the local file
b.commit().push(delocalize=True)

Context already bound to remote at s3://disdat-cdo-prd/
Pushed committed bundle None uuid 592e10eb-9ddf-4a89-a584-1929add7a284 to remote s3://disdat-cdo-prd/context


In [5]:
spark_master = 'spark://INTUL173d00074:7077'
app_name = "testapp"

In [6]:
api.apply("example-context", 
          pipelines.spark_tasks.RunSparkJob,
         params={'spark_master': spark_master,
                'app_name': app_name},
         incremental_push=True,
         force=True)

INFO: Informed scheduler that task   DriverTask_False______91f50a4619   has status   PENDING
INFO: Informed scheduler that task   RunSparkJob_testapp_s3_files_spark___INTUL173_a4a07736f6   has status   PENDING
INFO: Informed scheduler that task   ExternalDepTask__d41d8cd98f_d41d_592e10eb_9ddf_4a_988d52d43b   has status   DONE
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
INFO: [pid 85164] Worker Worker(salt=906261020, workers=1, host=INTUL173d00074, username=kyocum, pid=85164) running   RunSparkJob(spark_master=spark://INTUL173d00074:7077, app_name=testapp, input_bundle_name=s3_files)


Spark reading file:s3a://disdat-cdo-prd/context/example-context/objects/592e10eb-9ddf-4a89-a584-1929add7a284/file_1.txt
Lines with a: 0, lines with b: 0
['context/example-context/objects/9fb73578-89f8-4acd-b2a8-4fa29c3847d9/job_output/_SUCCESS', 'context/example-context/objects/9fb73578-89f8-4acd-b2a8-4fa29c3847d9/job_output/part-00000-ecef3b18-c21b-4351-b301-bdea2f6544fa-c000.snappy.parquet', 'context/example-context/objects/9fb73578-89f8-4acd-b2a8-4fa29c3847d9/job_output/part-00015-ecef3b18-c21b-4351-b301-bdea2f6544fa-c000.snappy.parquet']


INFO: [pid 85164] Worker Worker(salt=906261020, workers=1, host=INTUL173d00074, username=kyocum, pid=85164) done      RunSparkJob(spark_master=spark://INTUL173d00074:7077, app_name=testapp, input_bundle_name=s3_files)
INFO: Informed scheduler that task   RunSparkJob_testapp_s3_files_spark___INTUL173_a4a07736f6   has status   DONE
INFO: [pid 85164] Worker Worker(salt=906261020, workers=1, host=INTUL173d00074, username=kyocum, pid=85164) running   DriverTask(output_bundle=-, pipe_params={'spark_master': 'spark://INTUL173d00074:7077', 'app_name': 'testapp'}, pipe_cls=<class 'pipelines.spark_tasks.RunSparkJob'>, input_tags={}, output_tags={}, force=False)
INFO: [pid 85164] Worker Worker(salt=906261020, workers=1, host=INTUL173d00074, username=kyocum, pid=85164) done      DriverTask(output_bundle=-, pipe_params={'spark_master': 'spark://INTUL173d00074:7077', 'app_name': 'testapp'}, pipe_cls=<class 'pipelines.spark_tasks.RunSparkJob'>, input_tags={}, output_tags={}, force=False)
INFO: Inform

Pushed committed bundle None uuid 9fb73578-89f8-4acd-b2a8-4fa29c3847d9 to remote s3://disdat-cdo-prd/context


{'success': True, 'did_work': True}